<a href="https://colab.research.google.com/github/christiannaz2025/IC-2025/blob/main/Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Enviornment

In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [3]:
project_folder = '/content/drive/My Drive/Info_Challenge/Data and Code'
print(os.chdir(project_folder))# change directory
print(os.listdir(project_folder)) # list files in directory

None
['Lots and permissions.csv', 'Customer Cases.csv', 'Permits & Permissions.csv', 'Date table.csv', 'Parking Restrictions.csv', '***Processed_parking_data.csv', 'Special Events Construction.csv', 'merged_date_lots_permissions.csv', 'special_events_exploded.csv', 'lots_permissions_normalized.csv', 'l_d_merged_data.csv', 'Model_Main.ipynb', 'lots_permissions_with_valid_list.csv']


In [4]:
#import data into data frames

lots_permissions = pd.read_csv('Lots and permissions.csv')
parking_restrictions = pd.read_csv('Parking Restrictions.csv')
date_table = pd.read_csv('Date table.csv')
permits_permissions = pd.read_csv('Permits & Permissions.csv')

special_events_construction = pd.read_csv('Special Events Construction.csv')

Drop Uneeded columns from each table

# Join all tables

Normalize lots permission to join with date table

In [5]:
lots_permissions.fillna(0, inplace=True)

In [6]:
def replace_enforcement_days(day):
    if day == "Weekends":
        return ["Saturday", "Sunday"]
    elif day == "Weekdays":
        return ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    elif day == "Always":
        return ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    else:
        return day

lots_permissions['Enforcement Days'] = lots_permissions['Enforcement Days'].apply(replace_enforcement_days)

In [7]:
lots_permissions_normalized = lots_permissions.explode('Enforcement Days', ignore_index=True)
display(lots_permissions_normalized.head(20))

,Permits Type (Category),Physical Location (Yes/No),Parking Lot / Zone Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,Visitor After Hours,VM,W,WW,X,XX,Y,YC,YY,Z
0,Permitted,YES,A,Restricted after 4PM,Saturday,0:00:00,0:00:00,59,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,Permitted,YES,A,Restricted after 4PM,Sunday,0:00:00,0:00:00,59,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Permitted,YES,A,Restricted after 4PM,Monday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Permitted,YES,A,Restricted after 4PM,Tuesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Permitted,YES,A,Restricted after 4PM,Wednesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Permitted,YES,A,Restricted after 4PM,Thursday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Permitted,YES,A,Restricted after 4PM,Friday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Permitted,YES,A,Restricted after 4PM,Monday,16:00:00,7:00:00,60,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,Permitted,YES,A,Restricted after 4PM,Tuesday,16:00:00,7:00:00,60,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,Permitted,YES,A,Restricted after 4PM,Wednesday,16:00:00,7:00:00,60,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Create column with list of permissions (needed for later)

In [8]:
# Define which columns to check for permissions (all columns after the 7th column)
permission_cols = lots_permissions_normalized.columns[8:]  # Skip the first 8 columns which are metadata

# Function to get a list of valid permissions (columns with value 1.0)
def get_valid_permissions(row):
   valid_permissions = []
   for col in permission_cols:
       # Check if the value is 1.0 (or very close to 1.0 to account for floating point precision)
       if row[col] == 1.0 or (isinstance(row[col], float) and abs(row[col] - 1.0) < 0.001):
           valid_permissions.append(col)
   return valid_permissions

# Create a new column with lists of valid permissions
lots_permissions_normalized['Valid_Permissions'] = lots_permissions_normalized.apply(get_valid_permissions, axis=1)

display(lots_permissions_normalized.head(10))

,Permits Type (Category),Physical Location (Yes/No),Parking Lot / Zone Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,VM,W,WW,X,XX,Y,YC,YY,Z,Valid_Permissions
0,Permitted,YES,A,Restricted after 4PM,Saturday,0:00:00,0:00:00,59,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, All Campus, B, BB, C, CC..."
1,Permitted,YES,A,Restricted after 4PM,Sunday,0:00:00,0:00:00,59,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, All Campus, B, BB, C, CC..."
2,Permitted,YES,A,Restricted after 4PM,Monday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe..."
3,Permitted,YES,A,Restricted after 4PM,Tuesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe..."
4,Permitted,YES,A,Restricted after 4PM,Wednesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe..."
5,Permitted,YES,A,Restricted after 4PM,Thursday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe..."
6,Permitted,YES,A,Restricted after 4PM,Friday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe..."
7,Permitted,YES,A,Restricted after 4PM,Monday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,..."
8,Permitted,YES,A,Restricted after 4PM,Tuesday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,..."
9,Permitted,YES,A,Restricted after 4PM,Wednesday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,..."


join with permit and permissions

In [23]:
# Create a mapping from permission to associated permits
permission_to_permits = {}
for _, row in permits_permissions.iterrows():
   permission = row['Permissions']
   permit = row['Permit Full Name']

   if permission not in permission_to_permits:
       permission_to_permits[permission] = []
   permission_to_permits[permission].append(permit)

# Function to find associated permits for each list of valid permissions
def get_associated_permits(valid_permissions_list):
   associated_permits = []
   for permission in valid_permissions_list:
       if permission in permission_to_permits:
           associated_permits.extend(permission_to_permits[permission])
   return list(set(associated_permits))  # Remove duplicates

# Create a new column with lists of associated permits
lots_permissions_normalized['Associated_Permits'] = lots_permissions_normalized['Valid_Permissions'].apply(get_associated_permits)

display(lots_permissions_normalized['Associated_Permits'].head(10))



,Associated_Permits
0,"[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
1,"[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
2,"[All Campus, Service Permit (Paper), (2023-202..."
3,"[All Campus, Service Permit (Paper), (2023-202..."
4,"[All Campus, Service Permit (Paper), (2023-202..."
5,"[All Campus, Service Permit (Paper), (2023-202..."
6,"[All Campus, Service Permit (Paper), (2023-202..."
7,"[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
8,"[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
9,"[All Campus, (2023-2024) UMGC Permits, FAE (UM..."


,Permits Type (Category),Physical Location (Yes/No),Parking Lot / Zone Name,Posted Restrictions,Enforcement Days,Start Time - Daily,End Time - Daily,Count Valid Permissions in Lot by Date/Time,17FAE,18-19FAE,...,W,WW,X,XX,Y,YC,YY,Z,Valid_Permissions,Associated_Permits
0,Permitted,YES,A,Restricted after 4PM,Saturday,0:00:00,0:00:00,59,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, All Campus, B, BB, C, CC...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
1,Permitted,YES,A,Restricted after 4PM,Sunday,0:00:00,0:00:00,59,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, All Campus, B, BB, C, CC...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
2,Permitted,YES,A,Restricted after 4PM,Monday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
3,Permitted,YES,A,Restricted after 4PM,Tuesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
4,Permitted,YES,A,Restricted after 4PM,Wednesday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
5,Permitted,YES,A,Restricted after 4PM,Thursday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
6,Permitted,YES,A,Restricted after 4PM,Friday,7:00:00,16:00:00,8,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
7,Permitted,YES,A,Restricted after 4PM,Monday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
8,Permitted,YES,A,Restricted after 4PM,Tuesday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
9,Permitted,YES,A,Restricted after 4PM,Wednesday,16:00:00,7:00:00,60,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."


Inner join with date table, to identify for every date which what are the restrictions

In [24]:
# Standard merge (not cross merge)
l_d_merged_data = pd.merge(
    date_table,
    lots_permissions_normalized,
    left_on='Weekday',
    right_on='Enforcement Days',
    how='inner'
)

In [25]:
filtered_data = l_d_merged_data[l_d_merged_data['Date'] == '01/01/2024']
display(filtered_data)

,Date,Weekday,Week Number,Academic Year,Fiscal Year,Parking Restrictions - Simplified,Parking Restrictions - Specific,Special Restrictions (Start),Special Restrictions (End),Permits Type (Category),...,W,WW,X,XX,Y,YC,YY,Z,Valid_Permissions,Associated_Permits
0,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
1,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
2,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, AA, All Campus, Medical-FS, RR,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
3,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Special Restrictions,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,"[18-19FAE, 4H, A, AA, AE, All Campus, B, BB, C...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
4,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, AA, All Campus, B, BB, E, EE, H...","[All Campus, (2023-2024) UMGC Permits, Student..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
313,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
314,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
315,01/01/2024,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]


Convert special events table to merge with existing merged data

In [26]:
def create_date_range(start_date, end_date):
    """Create a list of all dates between start_date and end_date, inclusive"""
    if pd.isna(start_date) or pd.isna(end_date):
        return []

    try:
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)

        # Create a list of all dates in the range
        date_range = pd.date_range(start=start, end=end, freq='D')
        return [date.strftime('%Y-%m-%d') for date in date_range]
    except:
        return []

# Load the Special Events Construction data


# Convert Start Date and End Date to datetime for validation
special_events_construction['Start Date'] = pd.to_datetime(special_events_construction['Start Date'], errors='coerce')
special_events_construction['End Date'] = pd.to_datetime(special_events_construction['End Date'], errors='coerce')

# Create a new column with all dates between start and end
special_events_construction['All Dates'] = special_events_construction.apply(
    lambda row: create_date_range(row['Start Date'], row['End Date']),
    axis=1
)

# Explode the dataframe so each date gets its own row
special_events_exploded = special_events_construction.explode('All Dates', ignore_index=True)

# Rename the column for clarity
special_events_exploded = special_events_exploded.rename(columns={'All Dates': 'Date'})

# Drop rows with missing dates
special_events_exploded = special_events_exploded.dropna(subset=['Date'])
special_events_exploded.drop(columns=['Start Date', 'End Date'], inplace=True)

display(special_events_exploded.head(100))
special_events_exploded.to_csv('special_events_exploded.csv', index=False)

,Affected Lot/Populations,Days Impacted,Closure Type,Date
0,W,51,Construction,2023-01-03
1,W,51,Construction,2023-01-04
2,W,51,Construction,2023-01-05
3,W,51,Construction,2023-01-06
4,W,51,Construction,2023-01-07
...,...,...,...,...
95,UU,511,Construction,2024-02-21
96,UU,511,Construction,2024-02-22
97,UU,511,Construction,2024-02-23
98,UU,511,Construction,2024-02-24


Adjust merged dataset for Special Occasion

In [27]:
# Convert date formats to be consistent for comparison
special_events_exploded['Date'] = pd.to_datetime(special_events_exploded['Date'])
l_d_merged_data['Date'] = pd.to_datetime(l_d_merged_data['Date'])

# Iterate through each special event
for idx, event in special_events_exploded.iterrows():
    event_date = event['Date']
    affected_lot = event['Affected Lot/Populations']

    # Find all rows in the merged data that match this date
    matching_dates = l_d_merged_data['Date'] == event_date

    # If the affected lot column exists in the merged dataset, set its value to 0 for matching dates
    if affected_lot in l_d_merged_data.columns:
        l_d_merged_data.loc[matching_dates, affected_lot] = 0
        print(f"Set {affected_lot} to 0 for {event_date.strftime('%Y-%m-%d')} affecting {matching_dates.sum()} rows")
    else:
        print(f"Column {affected_lot} not found in merged dataset")

Set W to 0 for 2023-01-03 affecting 0 rows
Set W to 0 for 2023-01-04 affecting 0 rows
Set W to 0 for 2023-01-05 affecting 0 rows
Set W to 0 for 2023-01-06 affecting 0 rows
Set W to 0 for 2023-01-07 affecting 0 rows
Set W to 0 for 2023-01-08 affecting 0 rows
Set W to 0 for 2023-01-09 affecting 0 rows
Set W to 0 for 2023-01-10 affecting 0 rows
Set W to 0 for 2023-01-11 affecting 0 rows
Set W to 0 for 2023-01-12 affecting 0 rows
Set W to 0 for 2023-01-13 affecting 0 rows
Set W to 0 for 2023-01-14 affecting 0 rows
Set W to 0 for 2023-01-15 affecting 0 rows
Set W to 0 for 2023-01-16 affecting 0 rows
Set W to 0 for 2023-01-17 affecting 0 rows
Set W to 0 for 2023-01-18 affecting 0 rows
Set W to 0 for 2023-01-19 affecting 0 rows
Set W to 0 for 2023-01-20 affecting 0 rows
Set W to 0 for 2023-01-21 affecting 0 rows
Set W to 0 for 2023-01-22 affecting 0 rows
Set W to 0 for 2023-01-23 affecting 0 rows
Set W to 0 for 2023-01-24 affecting 0 rows
Set W to 0 for 2023-01-25 affecting 0 rows
Set W to 0 

In [28]:
display(l_d_merged_data)
Data_explode = l_d_merged_data.explode('Associated_Permits', ignore_index=True)
display(Data_explode)

,Date,Weekday,Week Number,Academic Year,Fiscal Year,Parking Restrictions - Simplified,Parking Restrictions - Specific,Special Restrictions (Start),Special Restrictions (End),Permits Type (Category),...,W,WW,X,XX,Y,YC,YY,Z,Valid_Permissions,Associated_Permits
0,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...","[All Campus, Service Permit (Paper), (2023-202..."
1,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[18-19FAE, 4H, A, AA, After Hours, All Campus,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
2,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, AA, All Campus, Medical-FS, RR,...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
3,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Special Restrictions,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,"[18-19FAE, 4H, A, AA, AE, All Campus, B, BB, C...","[All Campus, (2023-2024) UMGC Permits, FAE (UM..."
4,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, AA, All Campus, B, BB, E, EE, H...","[All Campus, (2023-2024) UMGC Permits, Student..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173285,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
173286,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
173287,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
173288,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]


,Date,Weekday,Week Number,Academic Year,Fiscal Year,Parking Restrictions - Simplified,Parking Restrictions - Specific,Special Restrictions (Start),Special Restrictions (End),Permits Type (Category),...,W,WW,X,XX,Y,YC,YY,Z,Valid_Permissions,Associated_Permits
0,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...",All Campus
1,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...",Service Permit (Paper)
2,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...",(2023-2024) UMGC Permits
3,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...",FAE (UMUC)
4,2024-01-01,Monday,28,2023-2024,2024,University-Observed Holiday,Winter Break; University-Observed Holiday,0,0,Permitted,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[18-19FAE, 4H, A, All Campus, Daily Service Pe...",Medical Permit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796269,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],NaN
796270,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],NaN
796271,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],NaN
796272,2025-08-24,Sunday,9,2024-2025,2025,"Intersemesters, Breaks, Exams, & Winter Term",Summer-Fall Intersemester,0,0,Metered Parking OR Special Restrictions,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],NaN
